<center><h1> DTS 302 LAB : Integrated Big Data Pipeline for Social Media Sentiment Analysis using Spark, 
MongoDB, and PostgreSQL</h1></center>

  <li>30007731 - Chukwukere Theresa - theresa.chukwukere@miva.edu.ng</li>
  <li>30078919 - Mayowa Ajasa-Lot - Mayowa.Ajasa-Lot@miva.edu.ng</li>
  <li>30007938 - Azeez Edunwale - azeez.edunwale@miva.edu.ng</li>
  <li>30007304 - Ihezukwu Kelechi Jesse - kelechukwu.ihezukwu@miva.edu.ng</li>
  <li>30007304 - Akpa Chidimma Anthonia - chidimma.akpa@miva.edu.ng</li>
  <li>30001238 - Abel Odemudia - abel.odemudia@miva.edu.ng</li>
  <li>30072709 - Amos Akintola, Adejesubisi - amos.adejesubisi@miva.edu.ng</li>
  <li>30082017 - Ogungbaigbe Oluwaseyi - oluwaseyi.ogungbaigbe@miva.edu.ng</li>
  <li>Cornelius Alobu</li>
   <li>EMMANUELLA AMPITAN</li>
    <li> RICHARD ITENE </li>
    <li> EBENEZER PHILIP</li>
</ol>


## Step 1: Twitter Data Collection (Tweepy + MongoDB)

In [ ]:
#  tweepy pymongo nltk pyspark pandas psycopg2-binary python-dotenv

import os
import sys
import json
from datetime import datetime
from typing import Optional, List

os.environ["PYSPARK_PYTHON"] = "python"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python"

# Load secrets from .env
try:
    from dotenv import load_dotenv
    load_dotenv()
    print("[OK] .env loaded")
except Exception as e:
    print("[WARN] Could not load .env:", e)

# Load Twitter, Mongo, Postgres from env
TWITTER_API_KEY = os.getenv("TWITTER_API_KEY")
TWITTER_API_SECRET = os.getenv("TWITTER_API_SECRET")
TWITTER_ACCESS_TOKEN = os.getenv("TWITTER_ACCESS_TOKEN")
TWITTER_ACCESS_SECRET = os.getenv("TWITTER_ACCESS_SECRET")
#TWITTER_BEARER_TOKEN = os.getenv("TWITTER_BEARER_TOKEN")
TWITTER_BEARER_TOKEN= "AAAAAAAAAAAAAAAAAAAAAFJY3gEAAAAAV2o3KZ8O6aca%2FGU9upEEgAWk5vk%3DSgyL4J9yeyNooICABmB3NeXF1F5cqg18dV7kvN5QdGVaKDxk0l" 
MONGO_URI = os.getenv("MONGO_URI")
MONGO_DB = os.getenv("MONGO_DB")
MONGO_COLLECTION = os.getenv("MONGO_COLLECTION")

POSTGRES_HOST = os.getenv("POSTGRES_HOST")
POSTGRES_PORT = os.getenv("POSTGRES_PORT")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")

print("[INFO] Loaded environment variables ")

# NLTK sentiment
try:
    import nltk
    nltk.download('vader_lexicon', quiet=True)
    from nltk.sentiment import SentimentIntensityAnalyzer
    print("[OK] NLTK VADER ready")
except Exception as e:
    print("[ERR] NLTK/VADER init failed:", e)

# Tweepy (Twitter), Mongo, Postgres
try:
    import tweepy
    from pymongo import MongoClient
    print("[OK] Tweepy & PyMongo imports")
except Exception as e:
    print("[ERR] Tweepy/PyMongo import:", e)

try:
    import psycopg2
    from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
    print("[OK] psycopg2 import")
except Exception as e:
    print("[ERR] psycopg2 import:", e)

# Spark
try:
    from pyspark.sql import SparkSession
    from pyspark.sql.functions import col, regexp_replace, lower, trim, to_timestamp, udf, explode, when, lit, array_distinct
    from pyspark.sql.types import StringType, ArrayType, StructType, StructField, TimestampType, IntegerType
    print("[OK] PySpark imports")
except Exception as e:
    print("[ERR] PySpark import:", e)


In [ ]:
## Fetch data from Twiter and Load to Mongo DB

In [2]:

from pyspark.sql import SparkSession

from pyspark.sql import SparkSession

os.environ["PYSPARK_PYTHON"] = "python"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python"

spark = SparkSession.builder \
    .appName("MongoDB Integration") \
    .config("spark.jars", r"C:\spark\mongo-spark-connector_2.13-10.5.0.jar,"
                            r"C:\spark\mongodb-driver-sync-4.9.0.jar,"
                            r"C:\spark\mongodb-driver-core-4.9.0.jar,"
                            r"C:\spark\bson-4.9.0.jar") \
    .getOrCreate()

# Check if spark is not None
if spark:
    print("Spark session successfully created!")
    print("Spark version:", spark.version)
else:
    print("Spark session not created.")


df = spark.read.format("mongodb") \
    .option("spark.mongodb.read.connection.uri", "mongodb://localhost:27017") \
    .option("spark.mongodb.read.database", "twitter_sentiment") \
    .option("spark.mongodb.read.collection", "tweets") \
    .load()

df.show(5, truncate=False)

Spark session successfully created!
Spark version: 4.0.0
+------------------------+-------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [17]:
import requests
from pymongo import MongoClient
import json
import os

# -----------------------------
# Load environment variables (optional if you have a .env)
# -----------------------------
# from dotenv import load_dotenv
# load_dotenv()
BEARER_TOKEN = os.getenv("TWITTER_BEARER_TOKEN")
#MONGO_URI = os.getenv("MONGO_URI")
#MONGO_DB = os.getenv("MONGO_DB")
#MONGO_COLLECTION = os.getenv("MONGO_COLLECTION")

# -----------------------------
# MongoDB setup
# -----------------------------
MONGO_URI = "mongodb://localhost:27017"
MONGO_DB = "twitter_sentiment"
MONGO_COLLECTION = "tweets"

client = MongoClient(MONGO_URI)
db = client[MONGO_DB]
tweets_collection = db[MONGO_COLLECTION]

# -----------------------------
# Twitter API setup
# -----------------------------

 
HEADERS = {
    "Authorization": f"Bearer {TWITTER_BEARER_TOKEN}",
    "User-Agent": "TwitterAPIv2"
}
SEARCH_URL = "https://api.twitter.com/2/tweets/search/recent"

# -----------------------------
# Function to fetch and save tweets
# -----------------------------
def fetch_and_save_tweets(query="technology", max_results=10):
    params = {
        "query": query,
        "max_results": max_results,
        "tweet.fields": ",".join([
            "id", "text", "created_at", "lang", "public_metrics",
            "entities", "source", "possibly_sensitive",
            "referenced_tweets", "context_annotations",
            "attachments", "geo"
        ]),
        "expansions": ",".join([
            "author_id", "in_reply_to_user_id", "referenced_tweets.id",
            "attachments.media_keys"
        ]),
        "user.fields": ",".join([
            "id", "username", "name", "verified",
            "description", "profile_image_url", "location",
            "public_metrics"
        ])
    }

    try:
        response = requests.get(SEARCH_URL, headers=HEADERS, params=params)
        response.raise_for_status()
    except requests.exceptions.HTTPError as e:
        print(f"Error fetching tweets: {e}")
        return
    except Exception as e:
        print(f"Unexpected error: {e}")
        return

    data = response.json()

    if "data" not in data:
        print("No tweets found.")
        return

    print(f"Fetched {len(data['data'])} tweets for keyword '{query}':\n")

    for tweet in data["data"]:
        # Include author info if available
        author_info = {}
        if "includes" in data and "users" in data["includes"]:
            for user in data["includes"]["users"]:
                if user["id"] == tweet.get("author_id"):
                    author_info = user
                    break

        tweet_doc = {
            "id": tweet.get("id"),
            "text": tweet.get("text"),
            "created_at": tweet.get("created_at"),
            "lang": tweet.get("lang"),
            "author_id": tweet.get("author_id"),
            "author": author_info,
            "retweet_count": tweet.get("public_metrics", {}).get("retweet_count", 0),
            "reply_count": tweet.get("public_metrics", {}).get("reply_count", 0),
            "like_count": tweet.get("public_metrics", {}).get("like_count", 0),
            "quote_count": tweet.get("public_metrics", {}).get("quote_count", 0),
            "entities": tweet.get("entities", {}),
            "source": tweet.get("source"),
            "possibly_sensitive": tweet.get("possibly_sensitive"),
            "referenced_tweets": tweet.get("referenced_tweets", []),
            "context_annotations": tweet.get("context_annotations", []),
            "attachments": tweet.get("attachments", {}),
            "geo": tweet.get("geo", {})
        }

        print(json.dumps(tweet_doc, indent=2))

        try:
            tweets_collection.insert_one(tweet_doc)
        except Exception as e:
            print(f"MongoDB insert error: {e}")

    print(f"\nSaved {len(data['data'])} tweets to MongoDB '{MONGO_DB}.{MONGO_COLLECTION}'.")

# -----------------------------
# Run the fetch
# -----------------------------
fetch_and_save_tweets(query="climate", max_results=10)



Fetched 10 tweets for keyword 'climate':

{
  "id": "1956352995110584505",
  "text": "@GoodLawProject Just waiting for climate warming to continue so they can grow grapes on Hadrian\u2019s Wall in Roman times. A period of massive expansion for the Roman Empire who struggled with the hot weather.",
  "created_at": "2025-08-15T13:51:15.000Z",
  "lang": "en",
  "author_id": "1363980218755674113",
  "author": {
    "username": "thepachyroom",
    "name": "@ThePachyRoom",
    "public_metrics": {
      "followers_count": 584,
      "following_count": 2081,
      "tweet_count": 42951,
      "listed_count": 1,
      "like_count": 1029,
      "media_count": 569
    },
    "verified": false,
    "location": "World",
    "id": "1363980218755674113",
    "profile_image_url": "https://pbs.twimg.com/profile_images/1878038312952160256/gRCf_2oT_normal.jpg",
    "description": "Facts have no feelings. Rule 1. Posts are my opinion. Rule 2. If posts contain misinfo refer to Rule 1. To think you must risk

## Get Insight to the Structure of the Data

In [18]:
df.printSchema()

root
 |-- _id: string (nullable = true)
 |-- attachments: struct (nullable = true)
 |    |-- media_keys: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- media_source_tweet_id: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- author: struct (nullable = true)
 |    |-- description: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- location: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- profile_image_url: string (nullable = true)
 |    |-- public_metrics: struct (nullable = true)
 |    |    |-- followers_count: integer (nullable = true)
 |    |    |-- following_count: integer (nullable = true)
 |    |    |-- tweet_count: integer (nullable = true)
 |    |    |-- listed_count: integer (nullable = true)
 |    |    |-- like_count: integer (nullable = true)
 |    |    |-- media_count: integer (nullable = true)
 |    |-- username: string (nullable = true)
 |    |-- verif

In [26]:
df.dtypes

[('_id', 'string'),
 ('attachments',
  'struct<media_keys:array<string>,media_source_tweet_id:array<string>>'),
 ('author',
  'struct<description:string,id:string,location:string,name:string,profile_image_url:string,public_metrics:struct<followers_count:int,following_count:int,tweet_count:int,listed_count:int,like_count:int,media_count:int>,username:string,verified:boolean>'),
 ('author_id', 'string'),
 ('context_annotations',
  'array<struct<domain:struct<id:string,name:string,description:string>,entity:struct<description:string,id:string,name:string>>>'),
 ('created_at', 'string'),
 ('entities',
  'struct<annotations:array<struct<start:int,end:int,probability:double,type:string,normalized_text:string>>,hashtags:array<struct<start:int,end:int,tag:string>>,mentions:array<struct<start:int,end:int,username:string,id:string>>,urls:array<struct<description:string,display_url:string,end:int,expanded_url:string,images:array<struct<url:string,width:int,height:int>>,media_key:string,start:int,

In [4]:
df_selected = df.select("_id", "author_id", "created_at", "id", "text")
df_selected.show(10)

+--------------------+-------------------+--------------------+-------------------+--------------------+
|                 _id|          author_id|          created_at|                 id|                text|
+--------------------+-------------------+--------------------+-------------------+--------------------+
|689c39ac4a7e21be2...|1850299875956559875|2025-08-13T07:07:...|1955526528135258172|RT @JeffNippard: ...|
|689c39ac4a7e21be2...|          203190690|2025-08-13T07:07:...|1955526516558926146|RT @IainGould: I ...|
|689c39ac4a7e21be2...|1776721164821229568|2025-08-13T07:07:...|1955526513845211144|With McCain, Romn...|
|689c39ac4a7e21be2...| 777858212775071745|2025-08-13T07:07:...|1955526513534935155|RT @BlockChain_lo...|
|689c39ac4a7e21be2...|1406590750117203973|2025-08-13T07:07:...|1955526502428315684|A good morning fr...|
|689c39ac4a7e21be2...|         2943487800|2025-08-13T07:07:...|1955526501866033261|RT @JamesWliso948...|
|689c39ac4a7e21be2...|           38647512|2025-08-13T07

##  Step 2 Data Cleaning Process 

In [19]:
from pyspark.sql.functions import col, expr

df_flat = df.select(
    col("author.username").alias("username"),
    col("text").alias("tweet"),
    col("created_at").alias("timestamp"),
   
)

df_flat.printSchema()
df_flat.show(5, truncate=False)

root
 |-- username: string (nullable = true)
 |-- tweet: string (nullable = true)
 |-- timestamp: string (nullable = true)

+--------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------+
|username      |tweet                                                                                                                                                                                                                                                                                                          |timestamp               |
+--------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [20]:
df_flat.filter(
    col("username").isNull() | col("tweet").isNull() | col("timestamp").isNull()
).show(10, truncate=False)

+--------+-----+---------+
|username|tweet|timestamp|
+--------+-----+---------+
+--------+-----+---------+



In [ ]:
# no missing value

In [21]:
from pyspark.sql.functions import when, col

# Replace null tweets with empty string
df_flat = df_flat.withColumn("tweet", when(col("tweet").isNull(), "").otherwise(col("tweet")))


In [22]:
def classify_sentiment(text):
    text = text.lower()
    positive_words = ["good", "great", "love", "happy", "excellent"]
    negative_words = ["bad", "terrible", "sad", "hate", "poor"]

    if any(word in text for word in positive_words):
        return "positive"
    elif any(word in text for word in negative_words):
        return "negative"
    else:
        return "neutral"

sentiment_udf = udf(classify_sentiment, StringType())
df_flat = df_flat.withColumn("sentiment", sentiment_udf(col("tweet")))

print(df_flat)



DataFrame[username: string, tweet: string, timestamp: string, sentiment: string]
